# Truffle Cultivation Analysis Example

This notebook demonstrates how to analyze simulation results and query the knowledge graph for insights.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import sys

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from simulation.simulator import TruffleCultivationSimulator, SimulationConfig
from scripts.query_knowledge_graph import KnowledgeGraphQuerier

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Run a Simulation

In [ ]:
# Create simulation configuration
config = SimulationConfig(
    total_time=6.0,  # 6 hours for quick demo
    dt=0.1,
    grid_size=(30, 30, 15),  # Small grid for speed
    grid_spacing=25.0,
    initial_tips=3,
    control_enabled=True,
    output_interval=0.5,
    output_dir="output/notebook_demo"
)

print(f"Running simulation: {config.total_time}h, grid {config.grid_size}")

# Run simulation
simulator = TruffleCultivationSimulator(config)
results = simulator.run()

print("Simulation completed!")

## 2. Analyze Simulation Results

In [ ]:
# Extract simulation data
output_data = results['output_data']
final_state = results['final_state']
stats = results['statistics']

print(f"Simulation Statistics:")
print(f"  Total steps: {stats['total_steps']}")
print(f"  Simulation time: {stats['simulation_time']:.2f} hours")
print(f"  Hyphal tips created: {stats['hyphal_tips_created']}")
print(f"  Active tips: {stats['hyphal_tips_active']}")
print(f"  Total hyphal length: {stats['total_hyphal_length']:.2f} μm")
print(f"  Control actions taken: {stats['control_actions_taken']}")

In [ ]:
# Create time series data
times = [step['time'] for step in output_data]
hyphal_tips = [step['abm_stats']['active_tips'] for step in output_data]
hyphal_length = [step['abm_stats']['total_length'] for step in output_data]
ph_values = [step['environmental_conditions']['pH'] for step in output_data]
ec_values = [step['environmental_conditions']['EC'] for step in output_data]
do_values = [step['environmental_conditions']['DO'] for step in output_data]
temp_values = [step['environmental_conditions']['temperature'] for step in output_data]

# Create plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Hyphal growth
axes[0, 0].plot(times, hyphal_tips, 'b-', linewidth=2, label='Active Tips')
axes[0, 0].set_xlabel('Time (hours)')
axes[0, 0].set_ylabel('Number of Hyphal Tips')
axes[0, 0].set_title('Hyphal Growth Over Time')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Total hyphal length
axes[0, 1].plot(times, hyphal_length, 'g-', linewidth=2, label='Total Length')
axes[0, 1].set_xlabel('Time (hours)')
axes[0, 1].set_ylabel('Total Hyphal Length (μm)')
axes[0, 1].set_title('Hyphal Network Growth')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Environmental conditions
axes[1, 0].plot(times, ph_values, 'r-', linewidth=2, label='pH')
axes[1, 0].plot(times, ec_values, 'b-', linewidth=2, label='EC')
axes[1, 0].set_xlabel('Time (hours)')
axes[1, 0].set_ylabel('pH / EC (mS/cm)')
axes[1, 0].set_title('Environmental Control')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Temperature and DO
ax2 = axes[1, 1].twinx()
line1 = axes[1, 1].plot(times, temp_values, 'orange', linewidth=2, label='Temperature')
line2 = ax2.plot(times, do_values, 'purple', linewidth=2, label='DO')
axes[1, 1].set_xlabel('Time (hours)')
axes[1, 1].set_ylabel('Temperature (°C)', color='orange')
ax2.set_ylabel('DO (mg/L)', color='purple')
axes[1, 1].set_title('Temperature and Dissolved Oxygen')
axes[1, 1].grid(True, alpha=0.3)

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
axes[1, 1].legend(lines, labels, loc='upper right')

plt.tight_layout()
plt.show()

## 3. Visualize Hyphal Density

In [ ]:
# Get final hyphal density
hyphal_density = np.array(final_state['hyphal_density'])

# Create 3D visualization
fig = plt.figure(figsize=(15, 5))

# XY plane (top view)
ax1 = fig.add_subplot(131)
xy_density = np.sum(hyphal_density, axis=2)
im1 = ax1.imshow(xy_density, cmap='viridis', origin='lower')
ax1.set_title('Hyphal Density - Top View (XY)')
ax1.set_xlabel('X (μm)')
ax1.set_ylabel('Y (μm)')
plt.colorbar(im1, ax=ax1, label='Density')

# XZ plane (side view)
ax2 = fig.add_subplot(132)
xz_density = np.sum(hyphal_density, axis=1)
im2 = ax2.imshow(xz_density, cmap='viridis', origin='lower')
ax2.set_title('Hyphal Density - Side View (XZ)')
ax2.set_xlabel('X (μm)')
ax2.set_ylabel('Z (μm)')
plt.colorbar(im2, ax=ax2, label='Density')

# YZ plane (front view)
ax3 = fig.add_subplot(133)
yz_density = np.sum(hyphal_density, axis=0)
im3 = ax3.imshow(yz_density, cmap='viridis', origin='lower')
ax3.set_title('Hyphal Density - Front View (YZ)')
ax3.set_xlabel('Y (μm)')
ax3.set_ylabel('Z (μm)')
plt.colorbar(im3, ax=ax3, label='Density')

plt.tight_layout()
plt.show()

print(f"Maximum hyphal density: {np.max(hyphal_density):.2f}")
print(f"Average hyphal density: {np.mean(hyphal_density):.2f}")
print(f"Total hyphal volume: {np.sum(hyphal_density):.2f}")

## 4. Analyze Nutrient Concentrations

In [ ]:
# Get nutrient concentration data
nutrient_data = final_state['nutrient_concentrations']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for i, (species, concentrations) in enumerate(nutrient_data.items()):
    if i >= 4:  # Only show first 4 species
        break
    
    row, col = i // 2, i % 2
    ax = axes[row, col]
    
    # Convert to numpy array
    conc_array = np.array(concentrations)
    
    # Show middle slice
    middle_slice = conc_array[:, :, conc_array.shape[2]//2]
    
    im = ax.imshow(middle_slice, cmap='plasma', origin='lower')
    ax.set_title(f'{species.capitalize()} Concentration')
    ax.set_xlabel('X (μm)')
    ax.set_ylabel('Y (μm)')
    plt.colorbar(im, ax=ax, label='Concentration (mol/m³)')

plt.tight_layout()
plt.show()

# Print concentration statistics
for species, concentrations in nutrient_data.items():
    conc_array = np.array(concentrations)
    print(f"{species.capitalize()}:")
    print(f"  Max: {np.max(conc_array):.4f} mol/m³")
    print(f"  Min: {np.min(conc_array):.4f} mol/m³")
    print(f"  Mean: {np.mean(conc_array):.4f} mol/m³")
    print(f"  Std: {np.std(conc_array):.4f} mol/m³")

## 5. Query Knowledge Graph (if available)

In [ ]:
try:
    # Try to connect to knowledge graph
    querier = KnowledgeGraphQuerier()
    
    print("Connected to knowledge graph!")
    
    # Query best protocols
    print("\nBest colonization protocols:")
    protocols = querier.get_best_colonization_protocols()
    for protocol in protocols[:5]:
        print(f"  - {protocol['protocol_name']}: {protocol['avg_colonization']:.1f}% colonization")
    
    # Query environmental conditions
    print("\nEnvironmental conditions for success:")
    conditions = querier.get_environmental_conditions_for_success(min_colonization=70.0)
    for condition in conditions[:5]:
        print(f"  - pH={condition['ph']:.2f}, EC={condition['ec']:.2f}, T={condition['temperature']:.1f}°C: "
              f"{condition['avg_colonization']:.1f}% colonization")
    
    querier.close()
    
except Exception as e:
    print(f"Could not connect to knowledge graph: {e}")
    print("Make sure Neo4j is running on localhost:7687")

## 6. Performance Analysis

In [ ]:
# Analyze control performance
control_actions = [step.get('control_actions', {}) for step in output_data]
performance_metrics = [step.get('performance_metrics', {}) for step in output_data]

# Extract control data
acid_doses = [actions.get('acid_dose', 0) for actions in control_actions]
base_doses = [actions.get('base_dose', 0) for actions in control_actions]
nutrient_doses = [actions.get('nutrient_dose', 0) for actions in control_actions]
oxygen_flows = [actions.get('oxygen_flow', 0) for actions in control_actions]
heating_powers = [actions.get('heating_power', 0) for actions in control_actions]

# Plot control actions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

axes[0, 0].plot(times, acid_doses, 'r-', linewidth=2, label='Acid Dose')
axes[0, 0].set_xlabel('Time (hours)')
axes[0, 0].set_ylabel('Acid Dose (L/min)')
axes[0, 0].set_title('pH Control - Acid Dosing')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(times, base_doses, 'b-', linewidth=2, label='Base Dose')
axes[0, 1].set_xlabel('Time (hours)')
axes[0, 1].set_ylabel('Base Dose (L/min)')
axes[0, 1].set_title('pH Control - Base Dosing')
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(times, nutrient_doses, 'g-', linewidth=2, label='Nutrient Dose')
axes[0, 2].set_xlabel('Time (hours)')
axes[0, 2].set_ylabel('Nutrient Dose (L/min)')
axes[0, 2].set_title('EC Control - Nutrient Dosing')
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(times, oxygen_flows, 'purple', linewidth=2, label='Oxygen Flow')
axes[1, 0].set_xlabel('Time (hours)')
axes[1, 0].set_ylabel('Oxygen Flow (L/min)')
axes[1, 0].set_title('DO Control - Oxygen Flow')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(times, heating_powers, 'orange', linewidth=2, label='Heating Power')
axes[1, 1].set_xlabel('Time (hours)')
axes[1, 1].set_ylabel('Heating Power (kW)')
axes[1, 1].set_title('Temperature Control - Heating')
axes[1, 1].grid(True, alpha=0.3)

# Total control effort
total_effort = [sum([acid_doses[i], base_doses[i], nutrient_doses[i], 
                    oxygen_flows[i], heating_powers[i]]) for i in range(len(times))]
axes[1, 2].plot(times, total_effort, 'black', linewidth=2, label='Total Effort')
axes[1, 2].set_xlabel('Time (hours)')
axes[1, 2].set_ylabel('Total Control Effort')
axes[1, 2].set_title('Total Control Effort')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate control statistics
print("Control Statistics:")
print(f"  Total acid dosed: {sum(acid_doses):.3f} L")
print(f"  Total base dosed: {sum(base_doses):.3f} L")
print(f"  Total nutrients dosed: {sum(nutrient_doses):.3f} L")
print(f"  Total oxygen flow: {sum(oxygen_flows):.3f} L")
print(f"  Total heating energy: {sum(heating_powers):.3f} kWh")
print(f"  Average control effort: {np.mean(total_effort):.3f}")

## 7. Summary and Insights

In [ ]:
print("=== SIMULATION SUMMARY ===")
print(f"Duration: {config.total_time} hours")
print(f"Grid size: {config.grid_size}")
print(f"Initial tips: {config.initial_tips}")
print(f"Final active tips: {stats['hyphal_tips_active']}")
print(f"Total hyphal length: {stats['total_hyphal_length']:.2f} μm")
print(f"Control actions taken: {stats['control_actions_taken']}")

print("\n=== FINAL ENVIRONMENTAL CONDITIONS ===")
final_env = final_state['environmental_conditions']
print(f"pH: {final_env['pH']:.2f}")
print(f"EC: {final_env['EC']:.2f} mS/cm")
print(f"DO: {final_env['DO']:.2f} mg/L")
print(f"Temperature: {final_env['temperature']:.2f} °C")
print(f"Flow rate: {final_env['flow_rate']:.2f} L/min")

print("\n=== KEY INSIGHTS ===")
print(f"• Hyphal network grew from {config.initial_tips} to {stats['hyphal_tips_active']} tips")
print(f"• Total hyphal length reached {stats['total_hyphal_length']:.2f} μm")
print(f"• Control system maintained stable conditions with {stats['control_actions_taken']} adjustments")
print(f"• Final pH of {final_env['pH']:.2f} is within optimal range for truffle cultivation")
print(f"• EC of {final_env['EC']:.2f} mS/cm indicates good nutrient availability")
print(f"• DO of {final_env['DO']:.2f} mg/L provides adequate oxygen for fungal growth")

print("\n=== RECOMMENDATIONS ===")
if final_env['pH'] < 6.0:
    print("• Consider increasing pH slightly for optimal mycorrhizal formation")
elif final_env['pH'] > 6.5:
    print("• Consider decreasing pH slightly for optimal mycorrhizal formation")
else:
    print("• pH is in optimal range for truffle cultivation")

if final_env['EC'] < 1.0:
    print("• Consider increasing nutrient concentration")
elif final_env['EC'] > 2.0:
    print("• Consider decreasing nutrient concentration to avoid osmotic stress")
else:
    print("• EC is in optimal range for nutrient uptake")

if stats['hyphal_tips_active'] < 10:
    print("• Consider longer simulation time or different initial conditions for more growth")
else:
    print("• Good hyphal growth achieved in simulation time")

print("\nSimulation analysis completed!")